# Box Loader Testing

I need to test a few things about the steps in the `argo_box_loader` notebook. There are a few things that might make a difference by decreasing the size of an array, but I want to try here on a box that takes a lot less time to load.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import argopy
import scipy.ndimage as filter
import scipy
import matplotlib
import gsw

import argopy
from argopy import DataFetcher as ArgoDataFetcher

argo_loader = ArgoDataFetcher(
    src="gdac", ftp="/swot/SUM05/dbalwada/Argo_sync", progress=True
)

In [2]:
import os
import sys
os.chdir('/home.ufs/amf2288/argo-intern/funcs')
sys.path.append(os.path.abspath('../funcs'))

import processing_funcs as prf

In [3]:
import dask
from dask import delayed
from dask.distributed import Client

client = Client()

In [4]:
#box1 = [-180,-179,0,1,0,2001]
box2 = [-180,-179,1,2,0,2001]
box3 = [-180,-179,2,3,0,2001]
box4 = [-180,-179,3,4,0,2001]
box5 = [-180,-179,4,5,0,2001]

In [31]:
# Example arguments for your function
boxes_list = [
    #(box1,2),
    (box2),
    (box3),
    (box4),
    (box5)
]

In [32]:
@dask.delayed
def get_box_delayed(*args, **kwargs):
    return prf.get_box(*args, **kwargs)

def get_box_dask(boxes_list, interp_step):
    
    boxes_list = [(box, interp_step) for box in boxes_list]
    
    # Create a list of delayed tasks
    tasks = [get_box_delayed(*args) for args in boxes_list]

    # Compute the results in parallel
    results = dask.compute(*tasks)

    # Process the results as needed
    for n,result in enumerate(results):
        result.to_netcdf("/swot/SUM05/amf2288/sync-boxes/test_boxes/lon:({},{})_lat:({},{})_ds_z.nc".format(boxes_list[n][0][0],boxes_list[n][0][1],boxes_list[n][0][2],boxes_list[n][0][3]))

In [33]:
get_box_dask(boxes_list, 2)

to xarray complete
point to profile complete
interpolation complete
adding spice complete
adding MLD complete
loading points complete


  0%|          | 0/50 [00:00<?, ?it/s]

to xarray complete
point to profile complete
interpolation complete
adding spice complete
adding MLD complete
loading points complete
to xarray complete
point to profile complete
interpolation complete
adding spice complete
adding MLD complete
loading points complete


  4%|▍         | 1/24 [00:01<00:39,  1.72s/it]

loading points complete


100%|██████████| 50/50 [06:03<00:00,  7.26s/it]


In [21]:
from typing import List, Tuple

#def generate_grid(min_lat: float, max_lat: float, min_lon: float, max_lon: float, step: float) -> List[Tuple[float, float, float, float]]:
def generate_grid(box, step):
    grid = []
    lon_min, lon_max, lat_min, lat_max = box[0],box[1],box[2],box[3]
    lat = lat_min
    while lat < lat_max:
        lon = lon_min
        while lon < lon_max:
            # Ensure the box does not exceed the specified bounds
            box_lat_max = min(lat + step, lat_max)
            box_lon_max = min(lon + step, lon_max)
            box = (lat, box_lat_max, lon, box_lon_max)
            grid.append(box)
            lon += step
        lat += step
    return grid

In [23]:
generate_grid([0,90,0,90,20], 20)

[(0, 20, 0, 20),
 (0, 20, 20, 40),
 (0, 20, 40, 60),
 (0, 20, 60, 80),
 (0, 20, 80, 90),
 (20, 40, 0, 20),
 (20, 40, 20, 40),
 (20, 40, 40, 60),
 (20, 40, 60, 80),
 (20, 40, 80, 90),
 (40, 60, 0, 20),
 (40, 60, 20, 40),
 (40, 60, 40, 60),
 (40, 60, 60, 80),
 (40, 60, 80, 90),
 (60, 80, 0, 20),
 (60, 80, 20, 40),
 (60, 80, 40, 60),
 (60, 80, 60, 80),
 (60, 80, 80, 90),
 (80, 90, 0, 20),
 (80, 90, 20, 40),
 (80, 90, 40, 60),
 (80, 90, 60, 80),
 (80, 90, 80, 90)]

In [24]:
generate_grid([0,20,0,20], 5)

[(0, 5, 0, 5),
 (0, 5, 5, 10),
 (0, 5, 10, 15),
 (0, 5, 15, 20),
 (5, 10, 0, 5),
 (5, 10, 5, 10),
 (5, 10, 10, 15),
 (5, 10, 15, 20),
 (10, 15, 0, 5),
 (10, 15, 5, 10),
 (10, 15, 10, 15),
 (10, 15, 15, 20),
 (15, 20, 0, 5),
 (15, 20, 5, 10),
 (15, 20, 10, 15),
 (15, 20, 15, 20)]

In [34]:
def get_target_box(area, region_step, target_step, interp_step):
    
    regions = generate_grid(area, region_step)
    
    for n,region in enumerate(regions):
        boxes = generate_grid(region, target_step)
        
        get_box_dask(boxes)
        
        text = "COMPLETED REGION {} OUT OF {}".format(n,len(regions))
        print('-' * len(text))
        print(text)
        print('-' * len(text))

In [9]:
cleint.close()

NameError: name 'cleint' is not defined